In [1]:
# from sklearn.feature_selection import SelectKBest, f_classif
# feature_cols = data.columns.drop('Target')
# # Keep 5 features
# selector = SelectKBest(f_classif, k=5)
# selector.fit(data[feature_cols], data["Target"])

# # Get the raw p-values for each feature, and transform from p-values into scores
# scores = -np.log10(selector.pvalues_)

# # Plot the scores.  See how "Pclass", "Sex", "Title", and "Fare" are the best?
# plt.bar(range(len(feature_cols)), scores)
# plt.xticks(range(len(feature_cols)), feature_cols, rotation='vertical')
# plt.show()

In [6]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import random
random.seed(10)
import data_pre as data_pre
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
data = pd.read_csv("C:/Santhosh/AIML/Github/Machine-Learning/Costa Rican Household Poverty Level Prediction/data/train.csv")

In [8]:
del_cols = ['idhogar', 'Id','dependency','tamhog','r4t3', 'tamviv', 'r4h1', 'hogar_total','r4h2', 'r4h3', 'r4m1', 'r4m2','r4m3', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'SQBescolari','SQBage','SQBhogar_total','SQBedjefe','SQBhogar_nin','SQBovercrowding','SQBdependency','SQBmeaned','agesq']

In [9]:
data_pre.drop_columns(data,del_cols)
data_pre.replce_na(data)
data_pre.get_cat_cols(data)
data_pre.encode_data(data)

In [11]:
from sklearn.feature_selection import chi2, SelectKBest
bestfeatures = SelectKBest(score_func=chi2, k=10)

feature_cols = data.columns.drop('Target')
fit = bestfeatures.fit(data[feature_cols],data['Target'])
dfscores = pd.DataFrame(fit.scores_)
dfscores.shape
dfcolumns = pd.DataFrame(feature_cols)
dfcolumns.shape
# #concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.shape
featureScores.head()
featureScores.columns = ['columns','Score']


In [12]:
data_selected = data[featureScores.nlargest(70,'Score')['columns']]

In [13]:
data_selected['Target'] = data['Target']

c:\users\katta\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
data_selected.shape

(9557, 71)

In [15]:
train, test = data_pre.train_test_split(data_selected)

In [17]:
X_train = train.drop(['Target'], axis =1)
Y_train = train['Target']
X_test = test.drop(['Target'], axis =1)
Y_test = test['Target']

In [22]:
from sklearn.neighbors import KNeighborsClassifier
test_scores = []
train_scores = []
for i in range(3,10):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,Y_train)
    
    train_scores.append(knn.score(X_train,Y_train))
    test_scores.append(knn.score(X_test,Y_test))

In [23]:
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

Max train score 81.4674339523934 % and k = [1]


In [24]:
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

Max test score 65.41077969649398 % and k = [4]
